This is the place where I experimented with things before commiting them to the package's code.

It's also where I put usage code, before I transform those to docs and/or tests.

# quotes_src

In [1]:
from saying.base import quotes_src
list(quotes_src)

['micheleriva_5421', 'englishquotesdatabase_75968']

In [2]:
d = quotes_src['micheleriva_5421']
len(d)

5421

In [4]:
d[:3]

[{'quote': 'Genius is one percent inspiration and ninety-nine percent perspiration.',
  'author': 'Thomas Edison'},
 {'quote': 'You can observe a lot just by watching.', 'author': 'Yogi Berra'},
 {'quote': 'A house divided against itself cannot stand.',
  'author': 'Abraham Lincoln'}]

In [5]:
dd = quotes_src['englishquotesdatabase_75968']
len(dd)

75968

In [6]:
dd[:3]

[['1', "'EXTRACTED DATA WITH HTTP:\\\\THEWEBMINER.COM'", "''", "'\\r'"],
 ['2', "'QUOTE'", "'AUTHOR'", "'GENRE\\r'"],
 ['3',
  "'Age is an issue of mind over matter. If you don't mind",
  "it doesn't matter.'",
  "'Mark Twain'",
  "'age\\r'"]]

# Prep data

## englishquotesdatabase_75968

In [9]:
from saying.base import quotes_src

dd = quotes_src['englishquotesdatabase_75968']
len(dd)

75968

In [7]:
dd[1], dd[2:4]

(['2', "'QUOTE'", "'AUTHOR'", "'GENRE\\r'"],
 [['3',
   "'Age is an issue of mind over matter. If you don't mind",
   "it doesn't matter.'",
   "'Mark Twain'",
   "'age\\r'"],
  ['4',
   "'Anyone who stops learning is old",
   "whether at twenty or eighty. Anyone who keeps learning stays young. The greatest thing in life is to keep your mind young.'",
   "'Henry Ford'",
   "'age\\r'"]])

In [24]:
from functools import partial
from saying.util import (
    remove_surrounding_single_quotes, clean_string, dict_of_rows_and_columns, map_fields
)


columns = list(map(lambda x: remove_surrounding_single_quotes(x).lower(), dd[1]))
ddd = list(dict_of_rows_and_columns(dd[2:], columns=columns))

ddd = [
    {clean_string(k): clean_string(v) for k, v in row.items()} for row in ddd
]

field_mapper = partial(
    map_fields({'quote': 'text', 'author': 'author', 'genre': 'category'})
)
ddd = list(map(field_mapper, ddd))

from saying.util import hash_text


def add_id(d: dict, id_func=lambda d: hash_text(d['text']), *, id_field='_id'):
    d['_id'] = id_func(d)
    return d

def iterable_to_dict_items(dicts, key_func=add_id):
    for d in dicts:
        yield key_func(d), d
    



quotes = list(map(add_id, ddd))



In [28]:
import pandas as pd

df = pd.DataFrame(quotes)
df

,text,author,category,_id
0,'Age is an issue of mind over matter. If you d...,it doesn't matter.',Mark Twain,f7596f22fafe66a99b30a3abf807ebd8
1,'Anyone who stops learning is old,whether at twenty or eighty. Anyone who keeps ...,Henry Ford,d157524b525aa2cb596f1c7d66d9e81d
2,Wrinkles should merely indicate where smiles h...,Mark Twain,age,7d69d57616991a78bebbad1bad03ea4c
3,True terror is to wake up one morning and disc...,Kurt Vonnegut,age,d26ca2ce975e45beb986f0cceb2115e0
4,A diplomat is a man who always remembers a wom...,Robert Frost,age,3b9e2c14673a8bb9b35f13b592fce46b
...,...,...,...,...
75961,'Look,there's no metaphysics on earth like chocolates.',Fernando Pessoa,8c4291f6956da81515a5c0caec2976d0
75962,Love is when the desire to be desired takes yo...,Henri de Toulouse-Lautrec,valentinesday,4d721a30e5d907fdfd32bcc51dd742d7
75963,I'm a laugh tart. I make no secret of that fact.,Hugh Grant,valentinesday,98fc9ba24726f6fa02017138c6300cd1
75964,I used to be a hopeless romantic - I fell in l...,Jeremy London,valentinesday,a60a9f66ed1f000b262b7cc1d3754385


In [30]:
def gather_by_id(df, min_count=2):
    """a dict whose keys are ids and values are the lists of text values
     with the same ids"""
    t = df.groupby('_id')['text'].apply(list).to_dict()
    return {k: v for k, v in t.items() if len(v) >= min_count}

t = gather_by_id(df)
len(t)

12148

In [38]:
it = iter(t.items())

In [50]:
k, q = next(it)
df[df['_id'] == k]

,text,author,category,_id
64208,'In sports,you simply aren't considered a real champion u...,Althea Gibson,006aa8429d0dfcc6da5e3004c2be0d01
64350,'In sports,people reach their peak very early. You have t...,but I'm still doing the work I always wanted t...,006aa8429d0dfcc6da5e3004c2be0d01
64355,'In sports,teams win and individuals don't.',Fran Tarkenton,006aa8429d0dfcc6da5e3004c2be0d01


In [51]:
ddd[64208]

{'text': "'In sports",
 'author': "you simply aren't considered a real champion until you have defended your title successfully. Winning it once can be a fluke winning it twice proves you are the best.'",
 'category': 'Althea Gibson',
 '_id': '006aa8429d0dfcc6da5e3004c2be0d01'}

In [86]:
sql_text = """
INSERT INTO `quotes`  (id_, quote, author, genre) VALUES

(64211,'She, and I', 'Alice', 'crypto\r'),
(64353,'Snap, crackle, and pop','Bob','cereal\r');
"""

from saying.util import extract_sql_data

t = extract_sql_data(sql_text)
tt = t[0]
tt

len(t[0])
t

sql_text = """
INSERT INTO `quotes`  (id_, quote, author, genre) VALUES

(64211,'She, and I', 'Alice', 'crypto\r'),
(64353,'Snap, crackle, and pop','Bob','cereal\r');
"""

table_name, columns, rows = extract_sql_data(sql_text)
expected_table_name = 'quotes'
expected_columns = ['id_', 'quote', 'author', 'genre']
expected_rows = [
    ['64211', "'She, and I'", "'Alice'", "'crypto\r')"],
    ['(64353', "'Snap, crackle, and pop'", "'Bob'", "'cereal\r'"]
]

[['64211',
  "'She",
  "and I'",
  "'Alice'",
  "'crypto\r')",
  '(64353',
  "'Snap",
  'crackle',
  "and pop'",
  "'Bob'",
  "'cereal\r'"]]

In [85]:
sql_text = """
INSERT INTO `quotes`  (id_, quote, author, genre) VALUES
(64211,'In sports, you simply aren\'t considered a real champion until you have defended your title successfully. Winning it once can be a fluke winning it twice proves you are the best.','Althea Gibson','sports\r'),
(64353,'In sports, people reach their peak very early. You have to move on. I don\'t know if I will ever surpass what I did at the Olympics, but I\'m still doing the work I always wanted to do.','Greg Louganis','sports\r'),
(64358,'In sports, teams win and individuals don\'t.','Fran Tarkenton','sports\r');
"""

from saying.util import extract_sql_data
import re


tt = extract_sql_data(sql_text)
# ttt = tt[0]
# len(ttt)
# print(*ttt, sep='\n')
# tt = sqlparse.split(t)
print(f"{len(tt)=}")
print(f"{len(tt[0])=}")
tt[0]




len(tt)=1
len(tt[0])=16


['64211',
 "'In sports",
 "you simply aren't considered a real champion until you have defended your title successfully. Winning it once can be a fluke winning it twice proves you are the best.'",
 "'Althea Gibson'",
 "'sports\r')",
 '(64353',
 "'In sports",
 "people reach their peak very early. You have to move on. I don't know if I will ever surpass what I did at the Olympics",
 "but I'm still doing the work I always wanted to do.'",
 "'Greg Louganis'",
 "'sports\r')",
 '(64358',
 "'In sports",
 "teams win and individuals don't.'",
 "'Fran Tarkenton'",
 "'sports\r'"]

In [20]:
from saying.util import hash_text

t = {hash_text(d['text']): d for d in ddd}
len(t)

58773

# Scrap - SQL parsing

In [90]:
import sqlparse

def extract_sql_data(sql_text):
    # Parse the SQL text
    parsed = sqlparse.parse(sql_text)[0]

    # Initialize variables
    table_name = ""
    columns = []
    rows = []

    # Process each token in the parsed SQL
    for token in parsed.tokens:
        # Extract table name from 'INSERT INTO' statement
        # if token.ttype is sqlparse.tokens.DML and token.value.upper() == 'INSERT':
        #     table_name = next(token for token in token.parent.tokens if isinstance(token, sqlparse.sql.Identifier)).get_name()

        if token.ttype is sqlparse.tokens.DML and token.value.upper() == 'INSERT':
            table_name_token = next((token for token in token.parent.tokens if isinstance(token, sqlparse.sql.Identifier)), None)
            if table_name_token is not None:
                table_name = table_name_token.get_name()
                
        # Extract column names
        if isinstance(token, sqlparse.sql.IdentifierList):
            columns = [identifier.get_name() for identifier in token.get_identifiers()]

        # Extract row values
        if isinstance(token, sqlparse.sql.Parenthesis):
            row_data = str(token).strip('()').split(',')
            # Clean up and format the row data
            row_data = [value.strip() for value in row_data]
            rows.append(tuple(row_data))

    return table_name, columns, rows

import sqlparse
import re

def extract_sql_data(sql_text):
    # Parse the SQL text
    parsed = sqlparse.parse(sql_text)[0]

    # Initialize variables
    table_name = ""
    columns = []
    rows = []

    # Manually extract table name and column names
    match = re.search(r"INSERT INTO `(.+?)`\s*\((.+?)\)", sql_text)
    if match:
        table_name = match.group(1)
        columns = [col.strip() for col in match.group(2).split(",")]

    # Process each token in the parsed SQL
    for token in parsed.tokens:
        # Extract row values
        if isinstance(token, sqlparse.sql.Parenthesis):
            row_data = str(token).strip('()').split(',')
            # Clean up and format the row data
            row_data = [value.strip() for value in row_data]
            rows.append(tuple(row_data))

    return table_name, columns, rows
    
# Example usage
sql_text = """
INSERT INTO `quotes`  (id_, quote, author, genre) VALUES

(64211,'She, and I', 'Alice', 'crypto\r'),
(64353,'Snap, crackle, and pop','Bob','cereal\r');
"""

table_name, columns, rows = extract_sql_data(sql_text)
expected_table_name = 'quotes'
expected_columns = ['id_', 'quote', 'author', 'genre']
expected_rows = [
    ('64211', "'She, and I'", "'Alice'", "'crypto\r')"),
    ('64353', "'Snap, crackle, and pop'", "'Bob'", "'cereal\r'")
]
# assert (table_name, columns, rows) == (expected_table_name, expected_columns, expected_rows)
table_name, columns, rows

('quotes', ['id_', 'quote', 'author', 'genre'], [])

In [96]:
parsed = sqlparse.parse(sql_text)[0]

# Initialize variables
table_name = ""
columns = []
rows = []

# Manually extract table name and column names
match = re.search(r"INSERT INTO `(.+?)`\s*\((.+?)\)", sql_text)
if match:
    table_name = match.group(1)
    columns = [col.strip() for col in match.group(2).split(",")]

# Process each token in the parsed SQL
for token in parsed.tokens:
    # Extract row values
    if isinstance(token, sqlparse.sql.Values):
        row_data = str(token).strip('()').split(',')
        # Clean up and format the row data
        row_data = [value.strip() for value in row_data]
        rows.append(tuple(row_data))

In [97]:
rows

[('VALUES\n\n(64211',
  "'She",
  "and I'",
  "'Alice'",
  "'crypto\r')",
  '(64353',
  "'Snap",
  'crackle',
  "and pop'",
  "'Bob'",
  "'cereal\r'")]

In [95]:
type(parsed.tokens[-2])

sqlparse.sql.Values

In [105]:
import pandas as pd

sql_text = """
INSERT INTO `quotes`  (id_, quote, author, genre) VALUES

(64211,'She, and I', 'Alice', 'crypto\r'),
(64353,'Snap, crackle, and pop','Bob','cereal\r');
"""

pd.read_sql(sql_text, con=None)

AttributeError: 'NoneType' object has no attribute 'cursor'

In [26]:
import sqlparse

def parse_sql_dump(sql_text):
    # Parse the SQL dump
    parsed = sqlparse.parse(sql_text)

    table_name = None
    columns_list = []
    rows = []

    for statement in parsed:
        if statement.get_type() == 'CREATE':
            # Extract table name
            tokens = [token for token in statement.tokens if not token.is_whitespace]
            for i, token in enumerate(tokens):
                if token.value.upper() == 'TABLE':
                    table_name = tokens[i + 1].get_real_name()
                elif token.ttype is sqlparse.tokens.Punctuation and token.value == '(':
                    # Extract columns
                    columns_list = [str(column).strip() for column in tokens[i + 1].get_identifiers()]
                    break

        elif statement.get_type() == 'INSERT':
            # Extract rows
            for token in statement.flatten():
                if token.value.upper().startswith('VALUES'):
                    values_str = token.value[6:].strip()
                    rows_str = values_str[1:-1]  # Remove surrounding parentheses
                    rows = [tuple(val.strip() for val in row.split(',')) for row in rows_str.split('),(')]

    return table_name, columns_list, rows

# Example usage
sql_text = """CREATE TABLE `quotes` (
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `quote` varchar(800) DEFAULT NULL,
  `author` varchar(100) DEFAULT NULL,
  `genre` varchar(100) DEFAULT NULL,
  PRIMARY KEY (`id`)
);
INSERT INTO `quotes` VALUES (1, 'Life is what happens...', 'John Lennon', 'life'), (2, 'Get busy living...', 'Stephen King', 'life');"""

from saying.base import quotes_src
from dol import Pipe
f = Pipe(*quotes_src.funcs['x16bkkamz6rkb78rzt7op_75968'].funcs[:-1])
sql_text = f()

table_name, columns, rows = parse_sql_dump(sql_text)

print("Table:", table_name)
print("Columns:", columns)
print("Rows:", rows)


Table: quotes
Columns: []
Rows: [('',)]


# Wikimedia quotes


In [1]:
from saying.wikimedia import download_and_process_wiki_data, parse_wikimedia_xml

In [2]:
download_and_process_wiki_data(('en', 'fr'))

DATE: 20231201
File ids.ttl already exists. Skipping download.

language='en'
File /Users/thorwhalen/.config/saying/wikimedia/en/wikidata.sql already exists. Skipping download.
File /Users/thorwhalen/.config/saying/wikimedia/en/pages.xml already exists. Skipping download.

language='fr'
File /Users/thorwhalen/.config/saying/wikimedia/fr/wikidata.sql already exists. Skipping download.
File /Users/thorwhalen/.config/saying/wikimedia/fr/pages.xml already exists. Skipping download.
Done.


In [11]:
from saying.util import extract_sql_data

# Example SQL text to be processed
from saying.util import extract_sql_data, extract_sql_table_rows

sql_text = """
/*!40000 ALTER TABLE `quotes` DISABLE KEYS */;
INSERT INTO `quotes` (col1,col2,col3) VALUES 
(1,'One', 1.0),
(2,'Two', 2.0),
(3,'Three', 3.0);
"""

# Test the function with the example SQL text
quotes_data = extract_sql_data(sql_text)

# Print the extracted data
print(quotes_data)

text = sql_text
import re
insert_statements = re.findall(
    (
        r"INSERT INTO "
        "`[^`]+"  # table name
        "`.+?"  # could be nothing, or colmns...
        "VALUES.+?\((.*?)\);"  # statement (rows)
    ),
    text,
    re.DOTALL,
)

AttributeError: module 'dol' has no attribute 'base'

In [10]:
import sqlparse

def parse_sql_dump(sql_text):
    # Parse the SQL dump
    parsed = sqlparse.parse(sql_text)

    table_name = None
    columns_list = []
    rows = []

    for statement in parsed:
        if statement.get_type() == 'CREATE':
            # Extract table name
            tokens = [token for token in statement.tokens if not token.is_whitespace]
            for i, token in enumerate(tokens):
                if token.value.upper() == 'TABLE':
                    table_name = tokens[i + 1].get_real_name()
                elif token.ttype is sqlparse.tokens.Punctuation and token.value == '(':
                    # Extract columns
                    columns_list = [str(column).strip() for column in tokens[i + 1].get_identifiers()]
                    break

        elif statement.get_type() == 'INSERT':
            # Extract rows
            for token in statement.flatten():
                if token.value.upper().startswith('VALUES'):
                    values_str = token.value[6:].strip()
                    rows_str = values_str[1:-1]  # Remove surrounding parentheses
                    rows = [tuple(val.strip() for val in row.split(',')) for row in rows_str.split('),(')]

    return table_name, columns_list, rows

# Example usage
sql_text = """CREATE TABLE `quotes` (
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `quote` varchar(800) DEFAULT NULL,
  `author` varchar(100) DEFAULT NULL,
  `genre` varchar(100) DEFAULT NULL,
  PRIMARY KEY (`id`)
);
INSERT INTO `quotes` VALUES (1, 'Life is what happens...', 'John Lennon', 'life'), (2, 'Get busy living...', 'Stephen King', 'life');
"""

table_name, columns, rows = parse_sql_dump(sql_text)
print("Table:", table_name)
print("Columns:", columns)
print("Rows:", rows)


Table: quotes
Columns: []
Rows: [('',)]


In [23]:
from saying.util import graze, graze_urls


b = graze(graze_urls['x16bkkamz6rkb78rzt7op_75968'])
print(f"{len(b)=}")

# table_name, columns, rows = parse_sql_dump(b.decode())
print("Table:", table_name)
print("Columns:", columns)
print(f"{len(rows)=}")

len(b)=13733779
Table: quotes
Columns: []
len(rows)=1


# XML parsing

In [2]:
import xml.etree.ElementTree as ET
import re

def parse_wikimedia_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    quotes_data = []

    for page in root.findall('{http://www.mediawiki.org/xml/export-0.10/}page'):
        title_elem = page.find('{http://www.mediawiki.org/xml/export-0.10/}title')
        text_elem = page.find('{http://www.mediawiki.org/xml/export-0.10/}revision/{http://www.mediawiki.org/xml/export-0.10/}text')

        # if title_elem is not None and text_elem is not None:
        #     title = title_elem.text
        #     text = text_elem.text

        #     # Basic parsing for quotes (may need adjustment based on the actual format)
        #     quotes = re.findall(r'\*\s*\'\'(.+?)\'\'', text)
        #     for quote in quotes:
        #         quotes_data.append((title, quote))

        if title_elem is not None and text_elem is not None and text_elem.text is not None:
            title = title_elem.text
            text = text_elem.text

            # Basic parsing for quotes (may need adjustment based on the actual format)
            quotes = re.findall(r'\*\s*\'\'(.+?)\'\'', text)
            for quote in quotes:
                quotes_data.append((title, quote))

    return quotes_data

# Example usage
file_path = '/Users/thorwhalen/.config/saying/wikiquotes/en/pages.xml'
quotes = parse_wikimedia_xml(file_path)
len(quotes)


221768

In [2]:
file_path = '/Users/thorwhalen/.config/saying/wikiquotes/en/pages.xml'
from dol import ValueCodecs

encode, decode = ValueCodecs.default.xml_etree
from pathlib import Path
# t = decode(Path(file_path).read_text())

# Scrap

In [38]:
from saying.util import hash_text

t = set(map(lambda x: hash_text(x['quote']), d))
tt = set(map(lambda x: hash_text(x['text']), dd))
len(t & tt)

1485

In [10]:
from dol import ValueCodecs, wrap_kvs
from dol.trans import ValueCodec

d = {}
dd = ValueCodecs.json(indent=True)(d)
dd['a'] = {'b': 2}
d


# ValueCodecs.default.json
# import json
# json.dumps


{'a': '{\n "b": 2\n}'}

[{'quote': 'Genius is one percent inspiration and ninety-nine percent perspiration.',
  'author': 'Thomas Edison'},
 {'quote': 'You can observe a lot just by watching.', 'author': 'Yogi Berra'},
 {'quote': 'A house divided against itself cannot stand.',
  'author': 'Abraham Lincoln'}]

In [27]:
dd = get_raw_data('micheleriva')
len(dd)

5421

In [24]:
from hashlib import md5
import re

non_alphanumeric_re = re.compile(r'\W+')

def lower_alphanumeric(text):
    return non_alphanumeric_re.sub(' ', text).strip().lower()

def hash_text(text):
    """Return a hash of the text, ignoring punctuation and capitalization.
    
    >>> (assert hash_text('Hello, world!') 
    ...     ==  hash_text('hello world') 
    ...     == '5eb63bbbe01eeed093cb22bb8f5acdc3'
    ... )

    """
    normalized_text = lower_alphanumeric(text)
    return md5(normalized_text.encode()).hexdigest()




'5eb63bbbe01eeed093cb22bb8f5acdc3'